# The Representation of an Algorithm Impacts its Maliciousness

In [12]:
import os
from pathlib import Path
from pprint import pprint
import subprocess

import torch
from torch import tensor

os.chdir("/home/lk3591/Documents/code/MalConv2")

from classifier import confidence_scores, get_model
from executable_helper import read_binary, _text_section_bounds_pefile

## Create and Test Diverse Executables

In [ ]:
WINDOWS = False

In [ ]:
compiler = "g++" if WINDOWS else "i686-w64-mingw32-g++"
source_code_files = [
	Path("one.cpp"),
	Path("two.cpp"),
	Path("three.cpp"),
]
compiler_flags = [
	[],
	[],
]
executables = []
for f in enumerate(source_code_files):
	for i, c in enumerate(compiler_flags):
		out = f"{f}_{i}.exe"
		executables.append(out)
		args = [compiler] + c + [f, "-o", out]
		subprocess.run(args)

## Initial Confidence Scores

In [11]:
def get_confidence_scores_from_files(files):
	confidences = []
	for f in files:
		binary = read_binary(f)
		x = tensor(binary, dtype=torch.int64)
		c = confidence_scores(model, x)
		confidences.append(c[0])
	return confidences

In [ ]:
model = get_model("gct")
confidences = get_confidence_scores_from_files(executables)
initial_confidences = {f : c for f, c in zip(executables, confidences)}
pprint(initial_confidences)

## Swap .text sections